# DATA 602 Assignment 12

## Kishore Prasad

## Non - Parellel Execution Output

In [15]:
%%timeit

import pandas as pd
import numpy as np

# Read Data
data = pd.read_csv("apple.2011.csv")
data.columns = ['Date','Price','Pert_change']

# Calculate mu and sigma
data_pert_change = pd.Series(data['Pert_change'][1:], dtype = float)
mu, sigma = np.mean(data_pert_change), np.std(data_pert_change)
mu, sigma

# simulate 10000 times
final_prices = np.empty(10000)

for i in range(0,10000):
    last_price = data['Price'][data['Price'].size - 1]
    random_percentages = np.random.normal(mu, sigma, 20)
    for j in random_percentages:
        last_price = last_price * (1 + j)

    final_prices[i] = last_price

# get Value at Risk
VaR = np.percentile(final_prices, 1)

1 loop, best of 3: 448 ms per loop


## Parallel Execution Output

In [16]:
import ipyparallel
clients = ipyparallel.Client()
clients.block = True
dv = clients.direct_view()

In [17]:
%%timeit

# Split the final_prices among the 4 processes
%px from numpy import empty
%px final_prices = empty(2500)
%px

import pandas as pd
import numpy as np

# Read Data
data = pd.read_csv("apple.2011.csv")
data.columns = ['Date','Price','Pert_change']

# Calculate mu and sigma
data_pert_change = pd.Series(data['Pert_change'][1:], dtype = float)
mu, sigma = np.mean(data_pert_change), np.std(data_pert_change)
mu, sigma

# simulate 10000 times
final_prices = np.empty(2500)

for i in range(0,2500):
    last_price = data['Price'][data['Price'].size - 1]
    random_percentages = np.random.normal(mu, sigma, 20)
    for j in random_percentages:
        last_price = last_price * (1 + j)

    final_prices[i] = last_price

%px

# Gather the output from the 4 processes
final_prices = dv.gather('final_prices')
final_prices = np.array(final_prices)

# get Value at Risk
VaR = np.percentile(final_prices, 1)


1 loop, best of 3: 380 ms per loop
